<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/LHMProgram/blob/main/Day3_GANs_DoneInClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt

#PyTorch Libraries
import torch
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import models,transforms
from torchvision.datasets import MNIST
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import torch.optim as optim
%matplotlib inline

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 16, 3, 2, 1)
    self.bn1 = nn.BatchNorm2d(16)
    self.conv2 = nn.Conv2d(16, 32, 3, 2, 1)
    self.bn2 = nn.BatchNorm2d(32)
    self.conv3 = nn.Conv2d(32, 64, 3, 2, 1)
    self.bn3 = nn.BatchNorm2d(64)

    self.linear1 = nn.Linear(64, 100)
    self.bn4 = nn.BatchNorm2d(100)
    self.linear1 = nn.Linear(100, 20)
    self.bn5 = nn.BatchNorm2d(20)
    self.linear1 = nn.Linear(20, 1)

  def forward(self, x):
    # (bs, 1, 28, 28)
    x = self.conv1(x)
    x = self.bn1(x)
    x = F.leaky_relu(x)
    # (bs, 16, 14, 14)
    x = self.conv2(x)
    x = self.bn2(x)
    x = F.leaky_relu(x)
    # (bs, 32, 7, 7)
    x = self.conv3(x)
    x = self.bn3(x)
    x = F.leaky_relu(x)
    # (bs, 64, 4, 4)
    print(x.shape)

    x = x.view(-1, 64*4*4)
    x = self.linear1(x)
    x = self.bn4(x)
    x = F.leaky_relu(x)

    x = self.linear2(x)
    x = self.bn5(x)
    x = F.leaky_relu(x)

    x = self.linear3(x)
    x = torch.sigmoid(x)
    return x

In [1]:
class Generator(nn.Module):
  def __init__(self, dz):
    self.linear1 = nn.Linear(dz, 20)
    self.bn1 = nn.BatchNorm1d(20)
    self.linear2 = nn.Linear(20, 100)
    self.bn2 = nn.BatchNorm1d(100)
    self.linear3 = nn.Linear(100, 64*4*4)
    self.bn3 = nn.BatchNorm1d(64*4*4)
    self.dconv1 = nn.ConvTransposed2d(64, 32, 3, 2, 1)
    self.bn4 = nn.BatchNorm1d(32)
    self.dconv2 = nn.ConvTransposed2d(32, 16, 3, 2, 1, output_padding = 1)
    self.bn5 = nn.BatchNorm1d(16)
    self.dconv3 = nn.ConvTransposed2d(16, 1, 3, 2, 1, output_padding = 1)

  def forward(self, x):
    x = self.linear1(x)
    x = self.bn1(x)
    x = F.leaky_relu(x)

    x = self.linear2(x)
    x = self.bn2(x)
    x = F.leaky_relu(x)

    x = self.linear3(x)
    x = self.bn3(x)
    x = F.leaky_relu(x)

    x = x.view(-1, 64, 4, 4)

    x = self.dconv1(x)
    x = self.bn4(x)
    x = F.leaky_relu(x)

    x = self.dconv2(x)
    x = self.bn5(x)
    x = F.leaky_relu(x)

    x = self.dconv3(x)
    x = F.tanh(x)
    return x

SyntaxError: ignored

In [ ]:
device = torch.device("cuda:0")
disc = Discriminator().to(device)
gen = Generator(32).to(device)

opt_d = optim.Adam(disc.parameters(), lr = 1e-4)
opt_g = optim.Adam(gen.parameters(), lr = 1e-4)

num_epochs = 3
for i in range(num_epochs):
  total_loss_gen = 0
  total_loss_disc = 0
  for batch in train_dataloader:
    # pass real images throught discriminator
    # pass fake images throught discriminator
    # create fake images throught discriminator and pass them throught the discriminator

    X = batch[0].to(device)
    labels = torch.ones(X.shape[0])
    out = disc(X)
    loss = F.bse_loss(out, labels)
    opt_d.zero_grad()
    loss.backward()

    noise = torch.randn((64, 32), device = device, dtype = torch.float)
    X_fake = gen(noise)
    out_fake = disc(X_fake)
    labels_fake = torch.zeros(X_fake.shape[0])
    loss_fake = F.bse_loss(out_fake, labels_fake)
    loss_fake.backward()
    opt_d.step()

    total_loss_gen += (loss.item() + loss_fake.item)

    # Generator

    noise = torch.randn((64, 32), device = device, dtype = torch.float)
    X_gfake = gen(noise)
    out_gfake = disc(X_gfake)
    labels_gfake = torch.ones(X_gfake.shape[0])
    loss_fake = F.bse_loss(out_gfake, labels_gfake)

    total_loss_gen += (loss.item() + loss_fake.item)

  ls_d.append(total_loss_disc)
  ls_g.append(total_loss_gen)